In [ ]:
print("sairam")

In [2]:
import os
print(os.getcwd())
os.chdir('../')
print(os.getcwd())

import pandas as pd 
import numpy as np

d:\AI\E2E-Wine-Quality-project\research
d:\AI\E2E-Wine-Quality-project


In [24]:
wines = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")
wines.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [26]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])   
        print('schema:', schema) 

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )
        
        return data_validation_config
        

In [27]:
from ml_project import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_colummns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    break
                else:
                    validation_status = True
                    
            with open(self.config.STATUS_FILE, "w") as file:
                        file.write(f"Validation status: {validation_status}")
            
            return validation_status        
            
        except Exception as e:
            logger.exception(e)
            raise e
            

In [28]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_colummns()
    
except Exception as e:
    print(e)

[2025-03-02 09:48:01,671: INFO: Yaml file config\config.yaml loaded successfully]
[2025-03-02 09:48:01,687: INFO: Yaml file params.yaml loaded successfully]
[2025-03-02 09:48:01,705: INFO: Yaml file schema.yaml loaded successfully]
[2025-03-02 09:48:01,708: INFO: Created directory at artifacts]
[2025-03-02 09:48:01,708: INFO: Created directory at artifacts/data_validation]
schema: {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64'}
